In [19]:
import findspark
findspark.init()
import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("portfolio1").getOrCreate()

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/

Using Python version 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023 14:38:34)
Spark context Web UI available at http://LAPTOP-93Q1EQTU:4042
Spark context available as 'sc' (master = local[*], app id = local-1696266336532).
SparkSession available as 'spark'.


In [20]:
#Task 1- Create a Dataframe using the commands given.
sample_data = sc.parallelize([
(1,"DD37Cf93aecA6Dc","Sheryl","Baxter","Rasmussen Group","East Leonard","Chile","229.077.5154","397.884.0519x718","zunigavanessa@smith.info","2020-08-24","http://www.stephenson.com/"),
(2,"1Ef7b82A4CAAD10","Preston","Lozano","Vega-Gentry","East Jimmychester","Djibouti","5153435776","686-620-1820x944","vmata@colon.com","2021-04-23","http://www.hobbs.com/"),
(3,"6F94879bDAfE5a6","Roy","Berry","Murillo-Perry","Isabelborough","Antigua and Barbuda","+1-539-402-0259","(496)978-3969x58947","beckycarr@hogan.com","2020-03-25","http://www.lawrence.com/"),
(4,"5Cef8BFA16c5e3c","Linda","Olsen","Dominguez, Mcmillan and Donovan","Bensonview","Dominican Republic","001-808-617-6467x12895","+1-813-324-8756","stanleyblackwell@benson.org","2020-06-02","http://www.good-lyons.com/"),
(5,"053d585Ab6b3159","Joanna","Bender","Rasmussen Group","West Priscilla","Slovakia (Slovak Republic)","001-234-203-0635x76146","001-199-446-3860x3486","colinalvarado@miles.net","2021-04-17","https://goodwin-ingram.com/")

])

sample_data_df = spark.createDataFrame(
sample_data
, [
"Index","Customer Id","First Name","Last Name","Company","City","Country","Phone 1","Phone 2","Email","Subscription Date","Website"
]
)
sample_data.take(1)
sample_data_df.take(1)
sample_data_df.show()
sample_data_df.printSchema()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|         Vega-Gentry|East Jimmychester|            Djibouti|          5153435776|    686-620-1820x944|     vmata@colon.com|     

In [21]:
#Task 2- Read Data from JSON and CSV files.

# Reading data from CSV
sample_data_csv = (
spark
.read
.csv(
'employees.csv'
, header=True
, inferSchema=True)
)
sample_data_csv.show()
sample_data_csv.printSchema()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|         Vega-Gentry|East Jimmychester|            Djibouti|          5153435776|    686-620-1820x944|     vmata@colon.com|     

In [31]:
#Reading data from JSON
sample_data_json_df = (
spark
.read
.option("multiline","true")
.option("mode", "PERMISSIVE")
.json('employees.json')
)
sample_data_json_df.printSchema()
sample_data_json_df.show()

root
 |-- City: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Customer Id: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Index: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Phone 1: string (nullable = true)
 |-- Phone 2: string (nullable = true)
 |-- Subscription Date: string (nullable = true)
 |-- Website\r: string (nullable = true)

+--------------------+---------------+-------------------+---------------+--------------------+----------+-----+---------+--------------------+--------------------+--------------------+--------------------+
|                City|        Company|            Country|    Customer Id|               Email|First Name|Index|Last Name|             Phone 1|             Phone 2|   Subscription Date|           Website\r|
+--------------------+---------------+-------------------+---------------+--------------------+---

In [23]:
# Task-3 Access underlying RDDS using relevant commands

sample_data_df.rdd.take(1)
sample_data.take(1)

import pyspark.sql as sql
import pyspark.sql.functions as f
sample_data_transformed = (
sample_data_df
.rdd
.map(lambda row: sql.Row(
**row.asDict()
, Email_UserName=row.Email.split('@')[0]
)
)
.map(lambda row: sql.Row(
**row.asDict()
, Email_Provider=row.Email.split('@')[1]
)
)
.toDF()
.select(
sample_data_df.columns +
[
'Email_Provider'
, 'Email_UserName'
])
)
sample_data_transformed.show()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|Email_Provider|  Email_UserName|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|    smith.info|   zunigavanessa|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|

In [24]:
#Task 4 Using Left Join SQL command to interact with the dataframes

company_df = sc.parallelize([
    ("Rasmussen Group", "Textile"),
    ("Murillo-Perry", "Service"),
    ("Dominguez, Mcmillan and Donovan", "Pharma"),
    ("Vega-Gentry", "Heavy Machinery")
]).toDF(["Company", "Industry"])
company_df.createOrReplaceTempView('industries')
sample_data_transformed.createOrReplaceTempView('employee')
spark.sql('''
SELECT a.*
, b.Industry
FROM employee AS a
LEFT JOIN industries AS b
ON a.Company==b.Company
ORDER BY Country DESC
''').show()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+---------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|Email_Provider|  Email_UserName|       Industry|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+---------------+
|    5|053d585Ab6b3159|    Joanna|   Bender|     Rasmussen Group|   West Priscilla|Slovakia (Slovak ...|001-234-203-0635x...|001-199-446-3860x...|colinalvarado@mil...|       2021-04-17|https://goodwin-i...|     miles.net|   colinalvara

In [25]:
#Task 5 Use Dataframe transformation commands and show the results obtained

sample_data_transformed.cache().show()


+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|Email_Provider|  Email_UserName|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|    smith.info|   zunigavanessa|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|

In [26]:
sample_data_transformed.coalesce(1).show()


+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|Email_Provider|  Email_UserName|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|    smith.info|   zunigavanessa|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|

In [27]:
sample_data_transformed.select('Company').distinct().show()


+--------------------+
|             Company|
+--------------------+
|     Rasmussen Group|
|         Vega-Gentry|
|       Murillo-Perry|
|Dominguez, Mcmill...|
+--------------------+



In [28]:
sample_data_transformed.dropDuplicates().show()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|Email_Provider|  Email_UserName|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|    smith.info|   zunigavanessa|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|

In [29]:

sample_data_transformed.dropna().show()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|Email_Provider|  Email_UserName|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+----------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|    smith.info|   zunigavanessa|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|